In [2]:
#Importamos la librería pandas que necesitamos para la lectura, conversión y limpieza de los datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import re 



# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

#Importamos librerías para localizar ruta archivos.
import os
import sys
sys.path.append("../")


pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
# pd.set_option('display.max_rows', None) # para poder visualizar todas las filas de los DataFrames

In [3]:
#Abrimos el csv y lo convertimos a un DataFrame.
df = pd.read_csv('finanzas-hotel-bookings_ORIGINAL.csv', index_col= 0)
df.head(10)

/Users/reyesaltozano/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,5,13,14,15,16,20,21,26,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
0,Resort Hotel,False,342.0,2015.0,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
1,Resort Hotel,False,737.0,NaN,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,NaN,NaN,Direct,0.0,0.0,0.0,NaN,C,4.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
2,Resort Hotel,False,7.0,2015.0,July,27.0,1.0,0.0,1.0,1.0,0.0,0.0,BB,GBR,NaN,Direct,0.0,0.0,0.0,A,C,0.0,NaN,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
3,Resort Hotel,False,13.0,NaN,July,27.0,1.0,0.0,1.0,1.0,NaN,0.0,BB,GBR,Corporate,Corporate,0.0,0.0,0.0,A,A,0.0,304.0,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
4,Resort Hotel,False,14.0,NaN,July,NaN,1.0,0.0,2.0,2.0,NaN,0.0,BB,NaN,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
5,Resort Hotel,False,14.0,NaN,July,27.0,1.0,0.0,2.0,2.0,NaN,0.0,BB,GBR,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,NaN,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
6,Resort Hotel,False,0.0,2015.0,July,NaN,1.0,0.0,2.0,2.0,NaN,0.0,BB,NaN,Direct,Direct,0.0,NaN,0.0,C,C,0.0,NaN,NaN,0.0,NaN,107.0,0.0,0.0,Check-Out,2015-07-03 00:00:00,NaN
7,Resort Hotel,False,9.0,2015.0,July,27.0,1.0,0.0,2.0,2.0,0.0,0.0,FB,NaN,Direct,Direct,0.0,NaN,0.0,C,C,0.0,303.0,NaN,0.0,NaN,103.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
8,Resort Hotel,True,85.0,2015.0,July,27.0,1.0,0.0,3.0,2.0,0.0,0.0,BB,PRT,NaN,NaN,0.0,NaN,0.0,A,A,0.0,240.0,NaN,0.0,Transient,82.0,0.0,1.0,Canceled,2015-05-06 00:00:00,NaN
9,Resort Hotel,True,75.0,2015.0,July,27.0,1.0,0.0,3.0,2.0,0.0,0.0,HB,NaN,NaN,TA/TO,0.0,0.0,0.0,D,D,0.0,15.0,NaN,0.0,Transient,105.5,0.0,0.0,Canceled,2015-04-22 00:00:00,NaN


In [4]:
# Utilizamos una función básica de exploración que hemos creado previamente

def exploracion_dataframe (dataframe):
    """
    Realiza un análisis exploratorio básico de un DataFrame, mostrando información sobre duplicados,
    valores nulos, tipos de datos, valores únicos para columnas categóricas y estadísticas descriptivas
    para columnas categóricas y numéricas, agrupadas por la columna de control.

    Parámetros:
    - dataframe (DataFrame): El DataFrame que se va a explorar.

    Returns: 
    No devuelve nada directamente, pero imprime en la consola la información exploratoria.
    """
    
    print(f"Número de filas: {dataframe.shape[0]}. Número de columnas: {dataframe.shape[1]}")
    print("\n ..................... \n")

    print(f"Número de duplicados: {dataframe.duplicated().sum()}")
    print("\n ..................... \n")
    
    print("Porcentaje de nulos:")
    df_nulos = pd.DataFrame(dataframe.isnull().sum() / dataframe.shape[0] * 100, columns = ["%_nulos"])
    display(df_nulos[df_nulos["%_nulos"] > 0])
    
    print("\n ..................... \n")
    print(f"Tipos de columnas:")
    display(pd.DataFrame(dataframe.dtypes, columns = ["tipo_dato"]))
    
    print("\n ..................... \n")
    print("Valores de las columnas categóricas: ")
    dataframe_categoricas = dataframe.select_dtypes(include = "O")
    
    for col in dataframe_categoricas.columns:
        print(f"La columna {col.upper()} tiene las siguientes valores únicos:")
        display(pd.DataFrame(dataframe[col].value_counts()))    
    
    print("\n ..................... \n")
    print("Valores de las columnas numéricas: ")
    dataframe_numericas = dataframe.select_dtypes(include='number')
    
    for col in dataframe_numericas.columns:
        print(f"La columna {col.upper()} tiene las siguientes valores únicos:")
        display(pd.DataFrame(dataframe[col].value_counts()))    

    print("\n ..................... \n")
    descripcion = dataframe.describe().T
    print(f"Principales valores estadísticos son:")
    print(descripcion.to_string())


In [5]:
df.head(10)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
0,Resort Hotel,False,342.0,2015.0,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
1,Resort Hotel,False,737.0,NaN,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,NaN,NaN,Direct,0.0,0.0,0.0,NaN,C,4.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
2,Resort Hotel,False,7.0,2015.0,July,27.0,1.0,0.0,1.0,1.0,0.0,0.0,BB,GBR,NaN,Direct,0.0,0.0,0.0,A,C,0.0,NaN,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
3,Resort Hotel,False,13.0,NaN,July,27.0,1.0,0.0,1.0,1.0,NaN,0.0,BB,GBR,Corporate,Corporate,0.0,0.0,0.0,A,A,0.0,304.0,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
4,Resort Hotel,False,14.0,NaN,July,NaN,1.0,0.0,2.0,2.0,NaN,0.0,BB,NaN,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
5,Resort Hotel,False,14.0,NaN,July,27.0,1.0,0.0,2.0,2.0,NaN,0.0,BB,GBR,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,NaN,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
6,Resort Hotel,False,0.0,2015.0,July,NaN,1.0,0.0,2.0,2.0,NaN,0.0,BB,NaN,Direct,Direct,0.0,NaN,0.0,C,C,0.0,NaN,NaN,0.0,NaN,107.0,0.0,0.0,Check-Out,2015-07-03 00:00:00,NaN
7,Resort Hotel,False,9.0,2015.0,July,27.0,1.0,0.0,2.0,2.0,0.0,0.0,FB,NaN,Direct,Direct,0.0,NaN,0.0,C,C,0.0,303.0,NaN,0.0,NaN,103.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
8,Resort Hotel,True,85.0,2015.0,July,27.0,1.0,0.0,3.0,2.0,0.0,0.0,BB,PRT,NaN,NaN,0.0,NaN,0.0,A,A,0.0,240.0,NaN,0.0,Transient,82.0,0.0,1.0,Canceled,2015-05-06 00:00:00,NaN
9,Resort Hotel,True,75.0,2015.0,July,27.0,1.0,0.0,3.0,2.0,0.0,0.0,HB,NaN,NaN,TA/TO,0.0,0.0,0.0,D,D,0.0,15.0,NaN,0.0,Transient,105.5,0.0,0.0,Canceled,2015-04-22 00:00:00,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182877 entries, 0 to 182876
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  object 
 2   lead_time                       119490 non-null  float64
 3   arrival_date_year               64829 non-null   float64
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        101004 non-null  float64
 6   arrival_date_day_of_month       119271 non-null  float64
 7   stays_in_weekend_nights         119490 non-null  float64
 8   stays_in_week_nights            119490 non-null  float64
 9   adults                          119428 non-null  float64
 10  children                        69302 non-null   float64
 11  babies                          119431 non-null  float64
 12  meal            

In [7]:
# Llamamos a la función para explorar el primer dataframe: df_activity

exploracion_dataframe(df)

Número de filas: 182877. Número de columnas: 32

 ..................... 

Número de duplicados: 63040

 ..................... 

Porcentaje de nulos:


,%_nulos
hotel,34.715683
is_canceled,34.715683
lead_time,34.661002
arrival_date_year,64.550490
arrival_date_month,34.715683
arrival_date_week_number,44.769435
arrival_date_day_of_month,34.780754
stays_in_weekend_nights,34.661002
stays_in_week_nights,34.661002
adults,34.694904



 ..................... 

Tipos de columnas:


,tipo_dato
hotel,object
is_canceled,object
lead_time,float64
arrival_date_year,float64
arrival_date_month,object
arrival_date_week_number,float64
arrival_date_day_of_month,float64
stays_in_weekend_nights,float64
stays_in_week_nights,float64
adults,float64



 ..................... 

Valores de las columnas categóricas: 
La columna HOTEL tiene las siguientes valores únicos:


...

La columna REQUIRED_CAR_PARKING_SPACES tiene las siguientes valores únicos:


,required_car_parking_spaces
0.0,111974
1.0,7383
2.0,28
3.0,3
8.0,2


La columna TOTAL_OF_SPECIAL_REQUESTS tiene las siguientes valores únicos:


,total_of_special_requests
0.0,70318
1.0,33226
2.0,12969
3.0,2497
4.0,340
5.0,40



 ..................... 

Principales valores estadísticos son:
                                   count         mean         std      min      25%       50%     75%          max
lead_time                       119490.0   104.172628  106.975949     0.00    18.00    69.000   161.0   737.000000
arrival_date_year                64829.0  2016.156196    0.706674  2015.00  2016.00  2016.000  2017.0  2017.000000
arrival_date_week_number        101004.0    27.175785   13.613871     1.00    16.00    28.000    38.0    53.000000
arrival_date_day_of_month       119271.0    15.795977    8.780503     1.00     8.00    16.000    23.0    31.000000
stays_in_weekend_nights         119490.0     0.939461    1.082472     0.00     0.00     1.000     2.0    19.914715
stays_in_week_nights            119490.0     2.515068    1.976511     0.00     1.00     2.000     3.0    50.000000
adults                          119428.0     6.244423   14.574814     0.00     2.00     2.000     2.0    59.000000
children        

In [8]:
duplicates = df.duplicated()
df[duplicates]

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
2191,Resort Hotel,True,170.0,2015.0,October,41.0,4.0,4.0,8.0,2.0,NaN,0.0,BB,CN,NaN,TA/TO,0.0,NaN,0.0,NaN,A,0.0,15.0,NaN,0.0,Transient,44.5,0.0,0.0,Canceled,2015-07-17 00:00:00,NaN
2630,Resort Hotel,True,148.0,2015.0,October,NaN,24.0,0.0,1.0,2.0,NaN,0.0,BB,NaN,NaN,Direct,0.0,0.0,0.0,A,A,0.0,NaN,NaN,0.0,Transient,47.0,0.0,0.0,Canceled,2015-06-26 00:00:00,NaN
2682,Resort Hotel,True,148.0,2015.0,October,43.0,24.0,0.0,1.0,1.0,0.0,0.0,BB,NaN,NaN,Direct,0.0,NaN,0.0,A,A,0.0,NaN,NaN,0.0,Transient,39.0,0.0,0.0,Canceled,2015-06-26 00:00:00,NaN
2812,Resort Hotel,True,155.0,2015.0,October,44.0,31.0,0.0,1.0,2.0,0.0,0.0,BB,NaN,Groups,Direct,0.0,NaN,0.0,A,A,0.0,NaN,NaN,0.0,Transient,36.0,0.0,0.0,Canceled,2015-10-26 00:00:00,NaN
2815,Resort Hotel,True,155.0,NaN,October,NaN,31.0,0.0,1.0,2.0,NaN,0.0,BB,PRT,Groups,Direct,0.0,NaN,0.0,NaN,A,0.0,NaN,NaN,0.0,Transient,36.0,0.0,0.0,Canceled,2015-10-26 00:00:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
duplicated_rows = df[df.duplicated(keep=False)]
duplicated_rows

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
2183,Resort Hotel,True,170.0,2015.0,October,41.0,4.0,4.0,8.0,2.0,NaN,0.0,BB,CN,NaN,TA/TO,0.0,NaN,0.0,NaN,A,0.0,15.0,NaN,0.0,Transient,44.5,0.0,0.0,Canceled,2015-07-17 00:00:00,NaN
2191,Resort Hotel,True,170.0,2015.0,October,41.0,4.0,4.0,8.0,2.0,NaN,0.0,BB,CN,NaN,TA/TO,0.0,NaN,0.0,NaN,A,0.0,15.0,NaN,0.0,Transient,44.5,0.0,0.0,Canceled,2015-07-17 00:00:00,NaN
2625,Resort Hotel,True,148.0,2015.0,October,NaN,24.0,0.0,1.0,2.0,NaN,0.0,BB,NaN,NaN,Direct,0.0,0.0,0.0,A,A,0.0,NaN,NaN,0.0,Transient,47.0,0.0,0.0,Canceled,2015-06-26 00:00:00,NaN
2630,Resort Hotel,True,148.0,2015.0,October,NaN,24.0,0.0,1.0,2.0,NaN,0.0,BB,NaN,NaN,Direct,0.0,0.0,0.0,A,A,0.0,NaN,NaN,0.0,Transient,47.0,0.0,0.0,Canceled,2015-06-26 00:00:00,NaN
2634,Resort Hotel,True,148.0,2015.0,October,43.0,24.0,0.0,1.0,1.0,0.0,0.0,BB,NaN,NaN,Direct,0.0,NaN,0.0,A,A,0.0,NaN,NaN,0.0,Transient,39.0,0.0,0.0,Canceled,2015-06-26 00:00:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df.duplicated().sum()

63040

In [11]:
df.drop_duplicates(inplace=True)

In [12]:
df.shape

(119837, 32)

In [13]:
print("Porcentaje de valores nulos en :")
print((df.isnull().sum() / len(df)) * 100)

Porcentaje de valores nulos en :
hotel                              1.528743
is_canceled                        1.528743
lead_time                          1.445297
arrival_date_year                 46.574931
arrival_date_month                 1.528743
arrival_date_week_number          16.831196
arrival_date_day_of_month          1.628045
stays_in_weekend_nights            1.445297
stays_in_week_nights               1.445297
adults                             1.505378
children                          42.916628
babies                             1.502875
meal                               1.528743
country                           46.384673
market_segment                    50.566186
distribution_channel              12.765673
is_repeated_guest                  5.711091
previous_cancellations            37.273129
previous_bookings_not_canceled     1.528743
reserved_room_type                34.746364
assigned_room_type                 1.528743
booking_changes                    1.511219

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119837 entries, 0 to 182876
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           118005 non-null  object 
 1   is_canceled                     118005 non-null  object 
 2   lead_time                       118105 non-null  float64
 3   arrival_date_year               64023 non-null   float64
 4   arrival_date_month              118005 non-null  object 
 5   arrival_date_week_number        99667 non-null   float64
 6   arrival_date_day_of_month       117886 non-null  float64
 7   stays_in_weekend_nights         118105 non-null  float64
 8   stays_in_week_nights            118105 non-null  float64
 9   adults                          118033 non-null  float64
 10  children                        68407 non-null   float64
 11  babies                          118036 non-null  float64
 12  meal            

In [15]:
def imputar_valores_iterative(file_path, columnas_a_imputar):

    # Crear una instancia de IterativeImputer
    imputer = IterativeImputer()
    
    # Imputar valores nulos solo en las columnas seleccionadas
    df[columnas_a_imputar] = imputer.fit_transform(df[columnas_a_imputar])
    
    # Guardar el DataFrame con valores imputados en un nuevo archivo (opcional)
    df.to_csv(file_path, index=False)
    
    print(f"Se han imputado los valores nulos en las columnas seleccionadas utilizando IterativeImputer. El archivo actualizado se ha guardado como '{file_path}'.")


# Columnas a las que se aplicará el IterativeImputer
columnas_a_imputar = ['lead_time', 'arrival_date_year', 'arrival_date_week_number','days_in_waiting_list' ,'adr']  # Reemplaza con los nombres reales de tus columnas

# Llamar a la función para imputar valores nulos
imputar_valores_iterative(file_path, columnas_a_imputar)


NameError: name 'file_path' is not defined

In [ ]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer  # Necesario para habilitar IterativeImputer
from sklearn.impute import IterativeImputer

def imputar_valores_iterative(file_path, columnas_a_imputar):
    # Cargar el archivo de datos en un DataFrame
  
    
    # Verificar si las columnas a imputar existen en el DataFrame
    columnas_existentes = [col for col in columnas_a_imputar if col in df.columns]
    if not columnas_existentes:
        print(f"No se encontraron las columnas especificadas en el archivo: {columnas_a_imputar}")
        return
    
    # Crear una instancia de IterativeImputer
    imputer = IterativeImputer()
    
    # Imputar valores nulos solo en las columnas seleccionadas
    df[columnas_existentes] = imputer.fit_transform(df[columnas_existentes])
    
    # Guardar el DataFrame con valores imputados en el mismo archivo
    df.to_csv(file_path, index=False)
    
    print(f"Se han imputado los valores nulos en las columnas seleccionadas utilizando IterativeImputer. El archivo actualizado se ha guardado como '{file_path}'.")


# Columnas a las que se aplicará el IterativeImputer
columnas_a_imputar = ['lead_time', 'arrival_date_year', 'arrival_date_week_number', 'days_in_waiting_list', 'adr']  # Reemplaza con los nombres reales de tus columnas

# Llamar a la función para imputar valores nulos
imputar_valores_iterative(file_path, columnas_a_imputar)


NameError: name 'file_path' is not defined